In [10]:
from supervised.automl import AutoML
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.class_weight import compute_sample_weight
import os
import sys
project_directory = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "."))
sys.path.append(project_directory)
from ml_cls.ml_base import MLBase

C:\Users\kafed\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
#загрузка и предобработка датасета
df = pd.read_csv(r'D:\PDResearch\experiments\feature_extraction\runs\2024-08-24_16-48-54\feature_dataset.csv')
features = ['VarFrq_PS','AvgFrqF_AU4_p11','DA_Surprise','AvgVopen_AU4_p11','DecFV_AU4_p11','DI_Happiness','VarVopen_PS','DI_Disgust','AvgVopen_PS','VarA_FT','AvgA_OC','AvgVopen_FT','NumA_PS','VarVclose_FT','AvgFrqF_AU12_p5','NumA_OC','DecA_FT','AvgVopen_OC','AvgVclose_AU12_p5','AvgFrq_FT','DA_Anger','Length_AU12_p5']
df['stage'] = df['stage'].replace({3.5:3, 2.5:3}).apply(int)
df = MLBase({}).fill_DI_healthy(df, features)

#можно не удалятть, тогда automl заполнит пропуски сам
df = df.dropna(subset = features)

In [ ]:
#Далее пример для обучения automl с warm, в примере без разделения на train и test, но разделение должно быть

In [ ]:
#для train set 
logo = LeaveOneGroupOut()
#logo = StratifiedGroupKFold(n_splits=8)
folds_array = list(logo.split(df[features], df['stage'].apply(int), groups = df['id']))
validation_strategy = { "validation_type": "custom",'folds': folds_array}
algorithms = [ 'CatBoost',  'LightGBM', 'Linear', 'Neural Network', 'Random Forest', 'Xgboost'] #['CatBoost', 'Extra Trees', ] #['Baseline',  'CatBoost', 'Decision Tree', 'Extra Trees', 'Nearest Neighbors', 'LightGBM', 'Linear', 'Neural Network', 'Random Forest', 'Xgboost']
#`logloss`, `f1`, `accuracy`
automl = AutoML(algorithms = algorithms,
               mode = "Perform", #"Perform", #"Compete"
               ml_task = "multiclass_classification",
               eval_metric = "accuracy",
               results_path = "D:\\PDResearch\\notebooks\\utils\\automl1\\", #path to output
               total_time_limit = 60 * 60 * 3,
                explain_level = 2,
                validation_strategy = validation_strategy)
df['class'] = df['stage'].apply(int)
df['sample_weight'] = compute_sample_weight(class_weight="balanced", y=df['class'])
automl.fit(df[features], df['class'].values, sample_weight=df['sample_weight'].values, cv = folds_array)

#для test set
automl.predict(df[features])